In [1]:
import pandas as pd

import sys
sys.executable

df = pd.read_csv('./data/reaugmented_3.csv')

df




,Pathogenic?,history_class,ethnicity,cancer_dx,known_brca,cancer_dx_type_1,cancer_dx_age_1,cancer_dx_type_2,cancer_dx_age_2,cancer_dx_type_3,...,rel_23_cancer_4,rel_23_cancer_5,rel_23_cancer_6,rel_23_cancer_7,rel_23_cancer_8,rel_23_cancer_9,rel_23_cancer_10,rel_23_age_1,rel_23_age_2,rel_23_age_3
0,0,3,6,1,-1,57,57,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
1,0,3,6,0,-1,32,32,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
2,0,3,2,1,-1,12,37,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
3,0,3,6,1,-1,40,55,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
4,0,3,6,1,-1,55,58,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46667,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
46668,0,0,6,0,-1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,81,-1,-1
46669,0,0,5,1,-1,13,22,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1
46670,0,0,6,0,0,0,-1,0,-1,0,...,0,0,0,0,0,0,0,-1,-1,-1


In [2]:
df2 = df.iloc[2213:, :]

In [3]:
from sklearn.model_selection import train_test_split

X=df2.loc[:, df.columns != 'history_class']  # Features
y=df2['history_class']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [3]:
from sklearn.model_selection import train_test_split

X=df.loc[:, df.columns != 'history_class']  # Features
y=df['history_class']  # Labels

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [4]:
X_train.shape

(16192, 332)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)

In [6]:
train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

In [7]:

train_loader = torch.utils.data.DataLoader(train, batch_size = 256, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = 256, shuffle = True)

In [8]:
class ANN(nn.Module):
    def __init__(self, input_dim = 332, output_dim = 4):
        super(ANN, self).__init__()
    
        # Input Layer (784) -> 784
        self.fc1 = nn.Linear(input_dim, 784)
        # 784 -> 128
        self.fc2 = nn.Linear(784, 128)
        # 128 -> 128
        self.fc3 = nn.Linear(128, 128)
        # 128 -> 64
        self.fc4 = nn.Linear(128, 64)
        # 64 -> 64
        self.fc5 = nn.Linear(64, 64)
        # 64 -> 32
        self.fc6 = nn.Linear(64, 32)
        # 32 -> 32
        self.fc7 = nn.Linear(32, 32)
        # 32 -> output layer(10)
        self.output_layer = nn.Linear(32,4)
        # Dropout Layer (20%) to reduce overfitting
        self.dropout = nn.Dropout(0.2)
    
    # Feed Forward Function
    def forward(self, x):
        
        # flatten image input
        
        # Add ReLU activation function to each layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        # Add dropout layer
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.dropout(x)
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = self.dropout(x)
        # Don't add any ReLU activation function to Last Output Layer
        x = self.output_layer(x)
        
        # Return the created model
        return x

In [9]:
model = ANN(input_dim = 332, output_dim = 4)

In [10]:
import torch.optim as optim
# specify loss function
loss_fn = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
import numpy as np
import torch
import torch.utils.data
from torch.autograd import Variable

In [12]:
epochs = 500

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

# Some lists to keep track of loss and accuracy during each epoch
epoch_list = []
train_loss_list = []
val_loss_list = []
train_acc_list = []
val_acc_list = []



# Start epochs
for epoch in range(epochs):
    # monitor training loss
    train_loss = 0.0
    val_loss = 0.0
    
    ###################
    # train the model #
    ###################
    # Set the training mode ON -> Activate Dropout Layers
    model.train() # prepare model for training
    # Calculate Accuracy         
    correct = 0
    total = 0
    
    # Load Train Images with Labels(Targets)
    for data, target in train_loader:
        
        # Convert our images and labels to Variables to accumulate Gradients
        data = Variable(data).float()
        target = Variable(target).type(torch.LongTensor)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # Calculate Training Accuracy 
        predicted = torch.max(output.data, 1)[1]        
        # Total number of labels
        total += len(target)
        # Total correct predictions
        correct += (predicted == target).sum()
        
        # calculate the loss
        loss = loss_fn(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
    
    # calculate average training loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    
    # Avg Accuracy
    accuracy = 100 * correct / float(total)
    
    # Put them in their list
    train_acc_list.append(accuracy)
    train_loss_list.append(train_loss)
    
        
    # Implement Validation like K-fold Cross-validation 
    # Set Evaluation Mode ON -> Turn Off Dropout
    model.eval() # Required for Evaluation/Test

    # Calculate Test/Validation Accuracy         
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Convert our images and labels to Variables to accumulate Gradients
            data = Variable(data).float()
            target = Variable(target).type(torch.LongTensor)

            # Predict Output
            output = model(data)

            # Calculate Loss
            loss = loss_fn(output, target)
            val_loss += loss.item()*data.size(0)
            # Get predictions from the maximum value
            predicted = torch.max(output.data, 1)[1]

            # Total number of labels
            total += len(target)

            # Total correct predictions
            correct += (predicted == target).sum()
    
    # calculate average training loss and accuracy over an epoch
    val_loss = val_loss/len(test_loader.dataset)
    accuracy = 100 * correct/ float(total)
    
    # Put them in their list
    val_acc_list.append(accuracy)
    val_loss_list.append(val_loss)
    
    # Print the Epoch and Training Loss Details with Validation Accuracy   
    print('Epoch: {} \tTraining Loss: {:.4f}\t Val. acc: {:.2f}%'.format(
        epoch+1, 
        train_loss,
        accuracy
        ))
    # save model if validation loss has decreased
    if val_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        val_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = val_loss
    # Move to next epoch
    epoch_list.append(epoch + 1)

Epoch: 1 	Training Loss: 0.8987	 Val. acc: 59.40%
Validation loss decreased (inf --> 0.826148).  Saving model ...
Epoch: 2 	Training Loss: 0.7757	 Val. acc: 64.17%
Validation loss decreased (0.826148 --> 0.739015).  Saving model ...
Epoch: 3 	Training Loss: 0.7121	 Val. acc: 65.95%
Validation loss decreased (0.739015 --> 0.708334).  Saving model ...
Epoch: 4 	Training Loss: 0.6759	 Val. acc: 67.54%
Validation loss decreased (0.708334 --> 0.660147).  Saving model ...
Epoch: 5 	Training Loss: 0.6358	 Val. acc: 68.35%
Validation loss decreased (0.660147 --> 0.602628).  Saving model ...
Epoch: 6 	Training Loss: 0.6116	 Val. acc: 69.14%
Epoch: 7 	Training Loss: 0.5960	 Val. acc: 71.48%
Epoch: 8 	Training Loss: 0.5858	 Val. acc: 71.48%
Validation loss decreased (0.602628 --> 0.595873).  Saving model ...
Epoch: 9 	Training Loss: 0.5593	 Val. acc: 73.68%
Validation loss decreased (0.595873 --> 0.545190).  Saving model ...
Epoch: 10 	Training Loss: 0.5559	 Val. acc: 72.11%
Epoch: 11 	Training L

Epoch: 116 	Training Loss: 0.2471	 Val. acc: 91.27%
Epoch: 117 	Training Loss: 0.2423	 Val. acc: 91.84%
Validation loss decreased (0.231461 --> 0.228858).  Saving model ...
Epoch: 118 	Training Loss: 0.2360	 Val. acc: 91.27%
Epoch: 119 	Training Loss: 0.2504	 Val. acc: 92.11%
Validation loss decreased (0.228858 --> 0.223991).  Saving model ...
Epoch: 120 	Training Loss: 0.2332	 Val. acc: 88.55%
Epoch: 121 	Training Loss: 0.2327	 Val. acc: 91.86%
Epoch: 122 	Training Loss: 0.2428	 Val. acc: 90.55%
Epoch: 123 	Training Loss: 0.2435	 Val. acc: 92.11%
Validation loss decreased (0.223991 --> 0.222957).  Saving model ...
Epoch: 124 	Training Loss: 0.2402	 Val. acc: 91.57%
Epoch: 125 	Training Loss: 0.2280	 Val. acc: 91.30%
Epoch: 126 	Training Loss: 0.2214	 Val. acc: 90.60%
Epoch: 127 	Training Loss: 0.2314	 Val. acc: 92.04%
Epoch: 128 	Training Loss: 0.2286	 Val. acc: 91.52%
Epoch: 129 	Training Loss: 0.2234	 Val. acc: 91.41%
Epoch: 130 	Training Loss: 0.2258	 Val. acc: 91.39%
Epoch: 131 	T

Epoch: 259 	Training Loss: 0.1973	 Val. acc: 91.66%
Epoch: 260 	Training Loss: 0.1982	 Val. acc: 90.26%
Epoch: 261 	Training Loss: 0.1977	 Val. acc: 92.94%
Epoch: 262 	Training Loss: 0.2024	 Val. acc: 93.34%
Validation loss decreased (0.204712 --> 0.204118).  Saving model ...
Epoch: 263 	Training Loss: 0.2057	 Val. acc: 91.88%
Epoch: 264 	Training Loss: 0.1984	 Val. acc: 92.22%
Epoch: 265 	Training Loss: 0.1948	 Val. acc: 92.65%
Validation loss decreased (0.204118 --> 0.203543).  Saving model ...
Epoch: 266 	Training Loss: 0.1914	 Val. acc: 92.76%
Epoch: 267 	Training Loss: 0.2080	 Val. acc: 91.97%
Epoch: 268 	Training Loss: 0.2014	 Val. acc: 92.65%
Epoch: 269 	Training Loss: 0.2033	 Val. acc: 89.97%
Epoch: 270 	Training Loss: 0.2036	 Val. acc: 92.56%
Epoch: 271 	Training Loss: 0.2137	 Val. acc: 92.94%
Epoch: 272 	Training Loss: 0.1926	 Val. acc: 93.30%
Epoch: 273 	Training Loss: 0.1879	 Val. acc: 92.51%
Epoch: 274 	Training Loss: 0.2163	 Val. acc: 93.12%
Validation loss decreased (0.2

Epoch: 406 	Training Loss: 0.1779	 Val. acc: 93.43%
Epoch: 407 	Training Loss: 0.1770	 Val. acc: 93.39%
Epoch: 408 	Training Loss: 0.1750	 Val. acc: 93.34%
Epoch: 409 	Training Loss: 0.1783	 Val. acc: 93.30%
Epoch: 410 	Training Loss: 0.1757	 Val. acc: 93.66%
Epoch: 411 	Training Loss: 0.1767	 Val. acc: 92.89%
Epoch: 412 	Training Loss: 0.1819	 Val. acc: 92.53%
Epoch: 413 	Training Loss: 0.1830	 Val. acc: 92.69%
Epoch: 414 	Training Loss: 0.1858	 Val. acc: 89.45%
Epoch: 415 	Training Loss: 0.1876	 Val. acc: 93.23%
Epoch: 416 	Training Loss: 0.1720	 Val. acc: 93.25%
Epoch: 417 	Training Loss: 0.1768	 Val. acc: 93.09%
Epoch: 418 	Training Loss: 0.1753	 Val. acc: 91.43%
Epoch: 419 	Training Loss: 0.1855	 Val. acc: 92.85%
Epoch: 420 	Training Loss: 0.1892	 Val. acc: 92.51%
Epoch: 421 	Training Loss: 0.1787	 Val. acc: 92.85%
Epoch: 422 	Training Loss: 0.1755	 Val. acc: 93.52%
Epoch: 423 	Training Loss: 0.1890	 Val. acc: 92.44%
Epoch: 424 	Training Loss: 0.1807	 Val. acc: 89.61%
Epoch: 425 	

In [14]:
df3 = df.loc[0:2212,:]

In [15]:
X2=df3.loc[:, df.columns != 'history_class']  # Features
y2=df3['history_class']  # Labels

In [16]:
X_test2 = torch.FloatTensor(X2.values)
y_test2 = torch.LongTensor(y2.values)

In [25]:
count = 0
for i in range(len(X_test2)):
    if y_test2[0]==torch.argmax(model(X_test2[i])):
        count+=1

In [27]:
count/(len(X_test2))

0.0022593764121102574